# Cleaning data

In [8]:
import json
import re
import emoji

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv("archive/SteamApps.csv")
temp = pd.read_csv("archive/steam_games.csv")
df = pd.concat([df, temp])
df

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,fullgame,...,controller_support,dlc,demos,achievements,reviews,recommendations,drm_notice,metacritic,alternate_appid,Unnamed: 0.1
0,0,demo,Pin Them Demo,1904630.0,0,True,NaN,NaN,NaN,"{'appid': '1764220', 'name': 'Pin Them'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,game,Al-Qadim: The Genie's Curse,1904640.0,0,False,"As an outcast, betrothed to the caliph's daugh...","As an outcast, betrothed to the caliph's daugh...",Experience the mysterious Al-Qadim game world ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,game,Dungeons & Dragons - Stronghold: Kingdom Simul...,1904650.0,0,False,Build and command your own kingdom in the Dung...,Build and command your own kingdom in the Dung...,Run your own kingdom in the legendary Dungeons...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,game,Chapel 3-D: The Ascent,1904680.0,0,False,"<h1>🔥 WISHLIST TO SURVIVE 🔥</h1><p><img src=""h...","<a href=""https://steamcommunity.com/linkfilter...","🔥 Chapel 3-D: The Ascent is a break-neck, viol...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,game,VTuber Gallery : Anime Pose,1904690.0,0,True,<strong>VTuber Gallery</strong> is <strong>#1 ...,<strong>VTuber Gallery</strong> is <strong>#1 ...,VTuber Gallery is #1 anime pose app that allow...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20554,270,video,The Chair,966800.0,13,False,<h1>Steam Video</h1><p>This content is only av...,Richard Sullivan has spent ten years on death ...,Richard Sullivan has spent ten years on death ...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270.0
20555,271,video,Head,966810.0,13,False,<h1>Steam Video</h1><p>This content is only av...,A group of 20-something puppets go on a weeken...,A group of 20-something puppets go on a weeken...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271.0
20556,272,video,The Barn,966820.0,13,False,<h1>Steam Video</h1><p>This content is only av...,It is Halloween 1989. Best friends Sam and Jos...,Halloween 1989. Sam and Josh are enjoying what...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.0
20557,273,video,Be My Cat: A Film for Anne,966830.0,13,False,<h1>Steam Video</h1><p>This content is only av...,Obsessed with convincing Hollywood actress Ann...,Obsessed with convincing Hollywood actress Ann...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,273.0


In [17]:
df.columns

Index(['Unnamed: 0', 'type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'fullgame', 'header_image', 'capsule_image', 'capsule_imagev5',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'developers', 'publishers', 'package_groups', 'platforms', 'categories',
       'release_date', 'support_info', 'background', 'background_raw',
       'content_descriptors', 'ratings', 'supported_languages', 'legal_notice',
       'price_overview', 'packages', 'genres', 'screenshots', 'movies',
       'ext_user_account_notice', 'controller_support', 'dlc', 'demos',
       'achievements', 'reviews', 'recommendations', 'drm_notice',
       'metacritic', 'alternate_appid', 'Unnamed: 0.1'],
      dtype='object')

In [17]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.drop(columns=["header_image", "capsule_image", "capsule_imagev5", "website",
                 "screenshots", "background", "background_raw", "content_descriptors",
                 "support_info", "about_the_game",
                 "detailed_description", "legal_notice", "movies", "ratings",
                 "ext_user_account_notice", "achievements", "reviews",
				 "controller_support", "dlc", "recommendations", "drm_notice",
				 "metacritic", "alternate_appid"], inplace=True)

# Some rows have the same index
df.reset_index(drop=True, inplace=True)
df.head(3)

,type,name,steam_appid,required_age,is_free,short_description,fullgame,pc_requirements,mac_requirements,linux_requirements,...,publishers,package_groups,platforms,categories,release_date,supported_languages,price_overview,packages,genres,demos
0,demo,Pin Them Demo,1904630.0,0,True,NaN,"{'appid': '1764220', 'name': 'Pin Them'}",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,...,['PRODUKTIVKELLER Studios'],[],"{'windows': 'True', 'mac': 'False', 'linux': '...","[{'id': 10, 'description': 'Game demo'}]","{'coming_soon': 'False', 'date': '16 Jun, 2023'}",NaN,NaN,NaN,NaN,NaN
1,game,Al-Qadim: The Genie's Curse,1904640.0,0,False,Experience the mysterious Al-Qadim game world ...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],...,['SNEG'],"[{'name': 'default', 'title': ""Buy Al-Qadim: T...","{'windows': 'True', 'mac': 'False', 'linux': '...","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': 'False', 'date': '29 Mar, 2022'}","English, French, German, Spanish - Spain","{'currency': 'VND', 'initial': 8000000, 'final...",[685932],"[{'id': '1', 'description': 'Action'}, {'id': ...",NaN
2,game,Dungeons & Dragons - Stronghold: Kingdom Simul...,1904650.0,0,False,Run your own kingdom in the legendary Dungeons...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],...,['SNEG'],"[{'name': 'default', 'title': 'Buy Dungeons & ...","{'windows': 'True', 'mac': 'False', 'linux': '...","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': 'False', 'date': '29 Mar, 2022'}","English, French, German","{'currency': 'VND', 'initial': 8000000, 'final...",[685935],"[{'id': '28', 'description': 'Simulation'}, {'...",NaN


## Get platforms and requirements

___

In [28]:
def true_false_string_to_json(x):
    if pd.isna(x):
        return {}
    try:
        return json.loads(x.replace("\'", "\"").replace("\"True\"", "true").replace("\"False\"", "false"))
    except:
        return json.loads(x.replace("\'", "\"").replace("True", "true").replace("False", "false"))


def simple_string_to_json(x, returning={}):
    if pd.isna(x):
        return returning
    else:
        return json.loads(x.replace("\'", "\""))


def string_with_apostrophe_to_json(x):
    if pd.isna(x):
        return {}
    else:
        try:
            temp = re.sub(r"\b'\B|\B'\b|\B'\B", "\"", x)
            return json.loads(temp)
        except:
            return {}

def string_with_apostrophe_to_json_2(x):
    if pd.isna(x):
        return {}
    else:
        try:
            x = x.replace("\"", "|")
            x = x.replace("\'", "\"")
            x = x.replace("|", "\'")
            return json.loads(x)
        except:
            return {}

In [29]:
df["platforms"].dropna().head(5)

0    {'windows': 'True', 'mac': 'False', 'linux': '...
1    {'windows': 'True', 'mac': 'False', 'linux': '...
2    {'windows': 'True', 'mac': 'False', 'linux': '...
3    {'windows': 'True', 'mac': 'False', 'linux': '...
4    {'windows': 'True', 'mac': 'False', 'linux': '...
Name: platforms, dtype: object

In [30]:
df["platforms"] = df["platforms"].apply(true_false_string_to_json)

In [31]:
df["platform_windows"] = df["platforms"].apply(lambda x: x.get("windows"))
df["platform_mac"] = df["platforms"].apply(lambda x: x.get("mac"))
df["platform_linux"] = df["platforms"].apply(lambda x: x.get("linux"))
df.drop(columns=["platforms"], inplace=True)

In [32]:
df.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'short_description', 'fullgame', 'pc_requirements', 'mac_requirements',
       'linux_requirements', 'developers', 'publishers', 'package_groups',
       'categories', 'release_date', 'supported_languages', 'price_overview',
       'packages', 'genres', 'demos', 'platform_windows', 'platform_mac',
       'platform_linux'],
      dtype='object')

In [33]:
df["pc_requirements"] = df["pc_requirements"].apply(string_with_apostrophe_to_json_2)
df["mac_requirements"] = df["mac_requirements"].apply(string_with_apostrophe_to_json_2)
df["linux_requirements"] = df["linux_requirements"].apply(string_with_apostrophe_to_json_2)

In [34]:
df.loc[10008, "pc_requirements"]

{'minimum': "<strong>Minimum:</strong><br><ul class='bb_ul'><li><strong>OS *:</strong> Windows 7<br></li><li><strong>Processor:</strong> i3<br></li><li><strong>Graphics:</strong> 1 GB</li></ul>",
 'recommended': "<strong>Recommended:</strong><br><ul class='bb_ul'><li><strong>OS:</strong> Windows 10<br></li><li><strong>Processor:</strong> i7<br></li><li><strong>Graphics:</strong> 4 GB</li></ul>"}

In [35]:
def handle_read_html(x, k, detail):
	soup = BeautifulSoup(x[k], "html.parser")
	require_list = soup.find("ul", class_="bb_ul")
	if require_list is None:
		return
	
	for e in require_list.find_all("li"):
		if e.find("strong") is None:
			continue
		if e.find("strong").get_text() == detail:
			return e.get_text()[(len(detail) + 1):]
	
	return None


def get_memory(x, detail="Memory:", recommended_priority=True):

	try:
		if recommended_priority:
			result = handle_read_html(x, "recommended", detail)
			if result is None:
				return handle_read_html(x, "minimum", detail)
		else:
			result = handle_read_html(x, "minimum", detail)
			if result is None:
				return handle_read_html(x, "recommended", detail)
		return result

	except:
		return None

In [36]:
df["storage"] = df["pc_requirements"].apply(lambda x: get_memory(x, "Storage:", False))
df["memory"] = df["pc_requirements"].apply(lambda x: get_memory(x, "Memory:", False))
df.drop(columns=["pc_requirements", "mac_requirements", "linux_requirements"], inplace=True)

In [37]:
df["memory"].unique()[:10]

array([None, '512 MB RAM', '1024 MB RAM', '8 GB RAM', '4 GB RAM',
       '16 GB RAM', '2 GB RAM', '6 GB RAM', '4 RAM MB RAM', '4096 MB RAM'],
      dtype=object)

In [41]:
num_compile = re.compile("(\d+[\.\,]?\d*)")


# Convert from string to int
def clean_storage(x):

	if pd.isna(x):
		return None
	
	global num_compile

	# Get number and bytes
	num = num_compile.search(x)
	if not num is None:
		
		type = ""
		if (x.find("GB") != -1) or (x.find("gb") != -1):
			type = "GB"
		elif (x.find("MB") != -1) or (x.find("mb") != -1):
			type = "MB"

		num = float(num.group(1).replace(",", "."))
		# Convert into megabytes
		if type == "":
			return None
		elif type == "GB":
			return num * 1024
		else:
			return num
	else:
		return None


df["storage_mb"] = df["storage"].apply(clean_storage)
df["memory_mb"] = df["memory"].apply(clean_storage)
df.drop(columns=["storage", "memory"], inplace=True)

## Get required age

___

In [43]:
df["required_age"].unique()

array([0, 18, 16, 12, 13, 17, 10, 15, 7, 3, 5, 171, 14, 6, 11, '0', '15',
       '3', '16', '17', '12', '18', '10', '11', '7', '13', '14', '18+', 1,
       4, '6', '1'], dtype=object)

In [44]:
def age_to_number(x):
    if isinstance(x, int):
        return x
    else:
        x = x.replace("+", "")
        return int(x)

df["required_age"] = df["required_age"].apply(age_to_number)

In [45]:
df.loc[df["required_age"] == 171, "required_age"] = 17

## Get release date

___

In [46]:
df["release_date"] = df["release_date"].apply(true_false_string_to_json)
df["release_date"]

0         {'coming_soon': False, 'date': '16 Jun, 2023'}
1         {'coming_soon': False, 'date': '29 Mar, 2022'}
2         {'coming_soon': False, 'date': '29 Mar, 2022'}
3           {'coming_soon': True, 'date': 'Coming soon'}
4         {'coming_soon': False, 'date': '21 Mar, 2022'}
                               ...                      
115965    {'coming_soon': False, 'date': '25 Oct, 2018'}
115966    {'coming_soon': False, 'date': '25 Oct, 2018'}
115967    {'coming_soon': False, 'date': '25 Oct, 2018'}
115968    {'coming_soon': False, 'date': '25 Oct, 2018'}
115969    {'coming_soon': False, 'date': '21 Dec, 2018'}
Name: release_date, Length: 115970, dtype: object

In [47]:
# Convert this column carefully
df.loc[df["steam_appid"] == 1904880, "release_date"]

14    {'coming_soon': True, 'date': '2024'}
Name: release_date, dtype: object

Convert to datetime would be absurd since some values only have years.

In [48]:
count = 0

def simple_check_datetime(x):
	try:
		pd.to_datetime(x)
		return None
	except:
		return x

df["release_date"].apply(lambda x: simple_check_datetime(x["date"])).unique()

array([None, 'Coming soon', 'To be announced', 'Q2 2024', 'Q3 2024',
       'Q1 2024', '2022年5月25日', 'Q4 2024', '13 сен. 2022', 'Q1 2025',
       'Q3 2029', 'Q2 2025', '19 gen 2023', 'Q4 2025', '2022 年 11 月 7 日',
       '17. Okt. 2022', 'Q4 2026', '2023 年 3 月 14 日', '11 мая. 2023',
       '4 апр. 2023', '28 фев. 2023', '8 ноя. 2022', 'Q3 2025',
       'Coming Soon', '25 мая. 2023', '22 ноя. 2023', '23 ENE 2024',
       '3.ᵉʳ trimestre del 2024', '4.º trimestre del 2024', 'Q2 2029',
       'Q1 2026', 'Q3 2026', '8 stycznia 2024', '2020年6月18日',
       '3 сен. 2020', '28/mai./2020', '2022年2月6日', '2022 年 2 月 3 日',
       '2022 年 1 月 27 日', '2 июн. 2023', '13. okt. 2021', '16 черв. 2022',
       '11 Eki 2016', '11/set./2017', '20 окт. 2016', '26/ago./2016',
       '4 mag 2016', '1 déc. 2015', '9 juin 2015', '13 Ağu 2013',
       '14 Ağu 2013', '18 Kas 2013', '20 Ara 2013', '14 Şub 2014',
       '8 авг. 2022', '23 ноя. 2015', '8. Dez. 2014', '2014 年 8 月 11 日',
       '2014 年 5 月 27 日', '8 AB

There are some valid dates but formatted in different languages. In this case, we will handle each case.

In [49]:
def get_year(x):
	if pd.isna(x):
		return None
	
	try:
		return re.findall("[0-9]{4}", x)[0]
	except Exception as e:
		if (x == "Coming soon") or (x == "Coming Soon") or (x == "To be announced") or (x == ""):
			return None
		raise Exception(x, e)


# Save this temporarily
df["release_date_temp"] = df["release_date"].apply(lambda x: x["date"])
df["release_year"] = df["release_date_temp"].apply(get_year)
df[["release_year", "release_date_temp"]].head()

,release_year,release_date_temp
0,2023,"16 Jun, 2023"
1,2022,"29 Mar, 2022"
2,2022,"29 Mar, 2022"
3,None,Coming soon
4,2022,"21 Mar, 2022"


In [50]:
quarter_1 = ["Q1", "Jan", "Feb", "Mar", "1 月", "3 月", "2月",
			 "фев.", "мая", "gen", "ENE", "stycznia", "Şub",
			 "мар", "2 月", "янв", "1月", "febr", "janv", "févr",
			 "jan", "mar", "MAR", "feb", "FEB"]
quarter_2 = ["Q2", "Apr", "May", "Jun", "5月", "6月", "mai",
			 "апр", "июн", "черв", "mag", "juin", "5 月",
			 "ABR", "4 月", "czerwca", "Juni", "พ.ค.", "Nis",
			 "4月", "jun", "apr", "MAY", "JUN"]
quarter_3 = ["Q3", "Jul", "Aug", "Sep", "сен", "3.ᵉʳ", "set",
			 "ago", "Ağu", "авг", "июл", "AGO", "sierpnia",
			 "8 月", "lug", "Eyl", "juil", "września", "9 月",
			 "JUL", "jul"]
quarter_4 = ["Q4", "Oct", "Nov", "Dec", "11 月", "Okt", "ноя",
			 "4.º", "okt", "Eki", "окт", "déc", "Kas", "Ara",
			 "Dez", "дек", "11 月", "Dez", "listopada", "NOV",
			 "dec", "nov", "NOV"]


def get_quarter(x):
	global quarter_1, quarter_2, quarter_3, quarter_4
	if pd.isna(x) or (x == "Coming soon") or (x == "Coming Soon") or (x == "To be announced") or (x == ""):
		return None
	elif any(qua in x for qua in quarter_1):
		return 1
	elif any(qua in x for qua in quarter_2):
		return 2
	elif any(qua in x for qua in quarter_3):
		return 3
	elif any(qua in x for qua in quarter_4):
		return 4
	else:
		return None


df["release_quarter"] = df["release_date_temp"].apply(get_quarter)
df[["release_year", "release_quarter"]].head()

,release_year,release_quarter
0,2023,2.0
1,2022,1.0
2,2022,1.0
3,None,NaN
4,2022,1.0


In [51]:
df["coming_soon"] = df["release_date"].apply(lambda x: x["coming_soon"])
df.drop(columns=["release_date_temp", "release_date"], inplace=True)

## Simplify the varibles

Including packages, developers, prices and discounts, etc.

___

In [18]:
df["short_description"] = df["short_description"].apply(lambda x: x.replace("&amp", "").replace("&quot", "") if pd.notna(x) else "")
df["short_description"] = df["short_description"].apply(lambda x: emoji.replace_emoji(x, ""))
df["short_description"] = df["short_description"].apply(lambda x: re.sub(r" +", " ", x))
df["short_description"]

0                                                          
1         Experience the mysterious Al-Qadim game world ...
2         Run your own kingdom in the legendary Dungeons...
3          Chapel 3-D: The Ascent is a break-neck, viole...
4         VTuber Gallery is #1 anime pose app that allow...
                                ...                        
115965    Richard Sullivan has spent ten years on death ...
115966    A group of 20-something puppets go on a weeken...
115967    Halloween 1989. Sam and Josh are enjoying what...
115968    Obsessed with convincing Hollywood actress Ann...
115969    ENTER the Forbidden World of SinVR. Fulfill yo...
Name: short_description, Length: 115970, dtype: object

In [52]:
df["package_groups"].dropna().head(20).to_list()

['[]',
 '[{\'name\': \'default\', \'title\': "Buy Al-Qadim: The Genie\'s Curse", \'description\': \'\', \'selection_text\': \'Select a purchase option\', \'save_text\': \'\', \'display_type\': 0, \'is_recurring_subscription\': \'false\', \'subs\': [{\'packageid\': 685932, \'percent_savings_text\': \' \', \'percent_savings\': 0, \'option_text\': "Al-Qadim: The Genie\'s Curse - 80.000₫", \'option_description\': \'\', \'can_get_free_license\': \'0\', \'is_free_license\': \'False\', \'price_in_cents_with_discount\': 8000000}]}]',
 "[{'name': 'default', 'title': 'Buy Dungeons & Dragons - Stronghold: Kingdom Simulator', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 685935, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'D&D Stronghold: Kingdom Simulator - 80.000₫', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': 'False', 'price_in_ce

In [54]:
def greedily_get_package_number(x):
	return x.count("'packageid'")

df["package_number"] = df["package_groups"].apply(greedily_get_package_number)
df.drop(columns=["package_groups"], inplace=True)
df["package_number"].unique()

array([ 0,  1,  4,  2,  3,  8, 13,  7,  5,  6,  9, 17], dtype=int64)

In [53]:
df["developers"].dropna().head()

0    ['PRODUKTIVKELLER Studios']
1          ['Cyberlore Studios']
2         ['Stormfront Studios']
3           ['Unfinished Games']
4                     ['Live3D']
Name: developers, dtype: object

In [56]:
def string_to_list(x, returning=[]):
	if pd.isna(x):
		return returning
	else:
		try:
			temp = x.replace("\', \'", "~|~")		\
					.replace("\", \'", "~|~")		\
					.replace("\', \"", "~|~")		\
					.replace("\"", "\'")			\
					.replace("~|~", "\", \"")		\
					.replace("[\'", "[\"")			\
					.replace("\']", "\"]")
			return json.loads(temp)
		except:
			print(x)
			print(temp, '\n')
			return returning


df["developers"] = df["developers"].apply(string_to_list)
df["publishers"] = df["publishers"].apply(string_to_list)

In [57]:
df["price_overview"].dropna().head(5)

1    {'currency': 'VND', 'initial': 8000000, 'final...
2    {'currency': 'VND', 'initial': 8000000, 'final...
5    {'currency': 'VND', 'initial': 7350000, 'final...
6    {'currency': 'VND', 'initial': 34900000, 'fina...
9    {'currency': 'VND', 'initial': 53500000, 'fina...
Name: price_overview, dtype: object

In [58]:
df["price_overview"] = df["price_overview"].apply(simple_string_to_json)

In [59]:
df["currency"] = df["price_overview"].apply(lambda x: x.get("currency", None))
df["initial_price"] = df["price_overview"].apply(lambda x: x.get("initial", None))
df["final_price"] = df["price_overview"].apply(lambda x: x.get("final", None))
df["discount_percent"] = df["price_overview"].apply(lambda x: x.get("discount_percent", None))
df.drop(columns=["price_overview"], inplace=True)

Count all developers and publishers

In [102]:
studio_list = []

for i in range(df.shape[0]):

	for dev in df.loc[i, "developers"]:
		if dev not in studio_list:
			studio_list.append(dev)

	for dev in df.loc[i, "publishers"]:
		if dev not in studio_list:
			studio_list.append(dev)


map_range = [i for i in range(len(studio_list))]
id_to_studio = dict(zip(map_range, studio_list))
studio_to_id = dict(zip(studio_list, map_range))
len(studio_list)

52949

In [103]:
with open("json_folder/studio_dict.json", "w", encoding="utf-8") as f:
	json_val = json.dumps(id_to_studio, ensure_ascii=False)
	print(json_val, file=f)

f.close()

In [109]:
df["developers"] = df["developers"].apply(lambda x: [studio_to_id[e] for e in x])
df["publishers"] = df["publishers"].apply(lambda x: [studio_to_id[e] for e in x])

df[["developers", "publishers"]]

,developers,publishers
0,[0],[0]
1,[1],[2]
2,[3],[2]
3,[4],[5]
4,[6],[6]
...,...,...
115965,[51418],[51418]
115966,"[51418, 52947]",[51418]
115967,"[51418, 52948]",[51418]
115968,[51418],[51418]


In [110]:
np.sort(df["developers"].apply(len).unique()), np.sort(df["publishers"].apply(len).unique())

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17,
        20, 41], dtype=int64),
 array([1, 2, 3, 4, 5, 6], dtype=int64))

In [111]:
df["developers_amount"] = df["developers"].apply(len)
df["publishers_amount"] = df["publishers"].apply(len)

## Get categories

Check if the game is single-player or multi-player, has VR support, or controller supp.

___

In [44]:
false_table = np.full((df.shape[0], 2), False)

player_table = pd.DataFrame(false_table, columns=["single", "multi"])
df = pd.concat([df, player_table], axis=1)

In [45]:
df["categories"] = df["categories"].apply(lambda x: simple_string_to_json(x, []))

In [46]:
player_mode_code = {
	"Single-player": "single",
	"Multi-player": "multi",
	"PvP": "multi",
	"Online PvP": "multi",
	"Shared/Split Screen PvP": "multi",
	"Cross-Platform Multiplayer": "multi",
	"Remote Play Together": "multi",
	"Co-op": "multi",
	"Online Co-op": "multi",
	"Shared/Split Screen Co-op": "multi",
	"LAN PvP": "multi",
	"MMO": "multi",
	"LAN Co-op": "multi"
}

count = 0
for i in range(df.shape[0]):
    for category in df.loc[i, "categories"]:
        key = player_mode_code.get(category["description"], "")
        if len(key) != "":
            df.loc[i, key] = True
    if count % 10000 == 0:
        print(f"Step {count} done")
    count += 1

df[["single", "multi"]]

Step 0 done
Step 10000 done
Step 20000 done
Step 30000 done
Step 40000 done
Step 50000 done
Step 60000 done
Step 70000 done
Step 80000 done
Step 90000 done
Step 100000 done
Step 110000 done


,single,multi
0,False,False
1,True,False
2,True,False
3,True,False
4,False,False
...,...,...
115965,False,False
115966,False,False
115967,False,False
115968,False,False


In [47]:
# Some games can have both singleplayer and multiplayer modes but they must have either one.
(df["single"] | df["multi"]).sum(), df.shape[0]

(102584, 115970)

In [48]:
df[~df["single"] & ~df["multi"]].head()

,type,name,steam_appid,required_age,is_free,fullgame,categories,supported_languages,packages,genres,...,package_number,currency,initial_price,final_price,discount_percent,developers_amount,publishers_amount,single,multi,
0,demo,Pin Them Demo,1904630.0,0,True,"{'appid': '1764220', 'name': 'Pin Them'}","[{'id': 10, 'description': 'Game demo'}]",NaN,NaN,NaN,...,0,None,NaN,NaN,NaN,1,1,False,False,True
4,game,VTuber Gallery : Anime Pose,1904690.0,0,True,NaN,[],English<strong>*</strong><br><strong>*</strong...,NaN,"[{'id': '51', 'description': 'Animation & Mode...",...,0,None,NaN,NaN,NaN,1,1,False,False,NaN
15,game,Restaurant Renovator Playtest,1904890.0,0,False,NaN,[],NaN,NaN,NaN,...,0,None,NaN,NaN,NaN,0,1,False,False,NaN
19,demo,STAMP Demo,1904990.0,0,True,"{'appid': '1827970', 'name': 'STAMP'}","[{'id': 10, 'description': 'Game demo'}]",NaN,NaN,NaN,...,0,None,NaN,NaN,NaN,2,1,False,False,True
20,game,SimVoiceAssistant,1905000.0,0,False,NaN,[],"English<strong>*</strong>, French<strong>*</st...",[686047],"[{'id': '28', 'description': 'Simulation'}, {'...",...,1,VND,7000000.0,7000000.0,0.0,1,1,False,False,NaN


Change the value: in that case, the game will be single-player

In [49]:
df.loc[df["single"] | df["multi"], "single"] = True

Controller and VR

In [50]:
false_table = np.full((df.shape[0], 2), False)

player_table = pd.DataFrame(false_table, columns=["support_vr", "support_controller"])
df = pd.concat([df, player_table], axis=1)

In [51]:
count = 0
for i in range(df.shape[0]):
    for category in df.loc[i, "categories"]:
        if category["description"].find("VR") != -1:
            df.loc[i, "support_vr"] = True
        elif category["description"].find("ontrol") != -1:   # Control, case-insensitive
            df.loc[i, "support_controller"] = True
    if count % 10000 == 0:
        print(f"Step {count} done")
    count += 1

df[["support_vr", "support_controller"]]

Step 0 done
Step 10000 done
Step 20000 done
Step 30000 done
Step 40000 done
Step 50000 done
Step 60000 done
Step 70000 done
Step 80000 done
Step 90000 done
Step 100000 done
Step 110000 done


,support_vr,support_controller
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
115965,False,True
115966,False,True
115967,False,True
115968,False,True


In [52]:
df.drop(columns=["categories"], inplace=True)

## Get languages

Selected languages: English, French, German, Spanish, Portuguese, Italian, Russian, Japanese, Chinese (both simplified and traditional), Korean, and Arabic.

___

In [53]:
def get_language_list(x):
    if pd.isna(x):
        return []
    x = x.replace("<strong>*</strong>", "")
    x = x.replace("<br>", "")
    x = x.replace("languages with full audio support", "")
    x = x.split(", ")
    return x

df["supported_languages"] = df["supported_languages"].apply(get_language_list)
df["supported_languages"].head()

0                                                   []
1           [English, French, German, Spanish - Spain]
2                            [English, French, German]
3    [English, French, German, Spanish - Spain, Por...
4                                            [English]
Name: supported_languages, dtype: object

In [54]:
with open("json_folder/language_to_code.json", encoding="utf8") as file:
    language_codes = file.read()
    language_codes = json.loads(language_codes)

In [55]:
false_table = np.full((df.shape[0], 11), False)

lang_table = pd.DataFrame(false_table, columns=["en", "fr", "de", "es", "po", "zh", "ja", "ko", "it", "ru", "ar"])
df = pd.concat([df, lang_table], axis=1)

In [56]:
count = 0
for i in range(df.shape[0]):
    for lang in df.loc[i, "supported_languages"]:
        key = language_codes.get(lang, "")
        if len(key) != "":
            df.loc[i, key] = True
    if count % 10000 == 0:
        print(f"Step {count} done")
    count += 1

df[["en", "fr", "de", "es", "po", "zh", "ja", "ko", "it", "ru", "ar"]]

Step 0 done
Step 10000 done
Step 20000 done
Step 30000 done
Step 40000 done
Step 50000 done
Step 60000 done
Step 70000 done
Step 80000 done
Step 90000 done
Step 100000 done
Step 110000 done


,en,fr,de,es,po,zh,ja,ko,it,ru,ar
0,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,False,False,False,False,False,False,False
2,True,True,True,False,False,False,False,False,False,False,False
3,True,True,True,True,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
115965,True,False,False,False,False,False,False,False,False,False,False
115966,True,False,False,False,False,False,False,False,False,False,False
115967,True,False,False,False,False,False,False,False,False,False,False
115968,True,False,False,False,False,False,False,False,False,False,False


In [57]:
df[["en", "fr", "de", "es", "po", "zh", "ja", "ko", "it", "ru", "ar"]].sum()

en    103255
fr     30944
de     31417
es     30279
po     19632
zh     30914
ja     26932
ko     17287
it     22665
ru     25726
ar      4501
dtype: int64

In [58]:
df.rename(columns={
    "en": "lang_en",
    "fr": "lang_fr",
    "de": "lang_de",
    "es": "lang_es",
    "po": "lang_po",
    "zh": "lang_zh",
    "ja": "lang_ja",
    "ko": "lang_ko",
    "it": "lang_it",
    "ru": "lang_ru",
    "ar": "lang_ar"
}, inplace=True)

In [59]:
df.drop(columns=["supported_languages"], inplace=True)

In [60]:
df.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'fullgame',
       'packages', 'genres', 'demos', 'platform_windows', 'platform_mac',
       'platform_linux', 'storage_mb', 'memory_mb', 'release_year',
       'release_quarter', 'coming_soon', 'package_number', 'currency',
       'initial_price', 'final_price', 'discount_percent', 'developers_amount',
       'publishers_amount', 'single', 'multi', '', 'support_vr',
       'support_controller', 'lang_en', 'lang_fr', 'lang_de', 'lang_es',
       'lang_po', 'lang_zh', 'lang_ja', 'lang_ko', 'lang_it', 'lang_ru',
       'lang_ar'],
      dtype='object')

## Extract genres

___

In [7]:
df["genres"] = df["genres"].apply(lambda x: simple_string_to_json(x, []))

In [62]:
genre_id_list = []
genre_name_list = []

for i in range(df.shape[0]):
    for genre in df.loc[i, "genres"]:
        if not genre["id"] in genre_id_list:
            genre_id_list.append(genre["id"])
            genre_name_list.append(genre["description"])

for id, name in zip(genre_id_list, genre_name_list):
	print(f"{id}: {name}")

1: Action
3: RPG
28: Simulation
2: Strategy
23: Indie
51: Animation & Modeling
53: Design & Illustration
55: Photo Editing
57: Utilities
59: Web Publishing
70: Early Access
25: Adventure
4: Casual
37: Free to Play
9: Racing
18: Sports
29: Massively Multiplayer
54: Education
60: Game Development
58: Video Production
52: Audio Production
56: Software Training
50: Accounting
80: Movie
73: Violent
74: Gore
81: Documentary
82: Episodic
72: Nudity
71: Sexual Content
83: Short
84: Tutorial


In [63]:
genre_dict = {}

for id, name in zip(genre_id_list, genre_name_list):
	genre_dict[id] = name

with open("json_folder/genre_dict.json", "w") as f:
	json_val = json.dumps(genre_dict)
	print(json_val, file=f)

In [64]:
def get_genre_list(x):
	game_list = []
	for genre in x:
		game_list.append(int(genre["id"]))
	return game_list

df["genres"] = df["genres"].apply(get_genre_list)

In [65]:
false_table = np.full((df.shape[0], 3), False)

genre_table = pd.DataFrame(false_table, columns=["tool", "nsfw", "film"])
df = pd.concat([df, genre_table], axis=1)

genre_code = {
	50: "tool",			# Accounting
	51: "tool",			# Animation & Modeling
	52: "tool",			# Audio Production
	53: "tool",			# Design & Illustration
	54: "tool",			# Education
	55: "tool",			# Photo Editing
	56: "tool",			# Software Training
	57: "tool",			# Utilities
	58: "tool",			# Video Production
	59: "tool",			# Web Publishing
	
	71: "nsfw",			# Sexual Content
	72: "nsfw",			# Nudity
	73: "nsfw",			# Violent
	74: "nsfw",			# Gore

	80: "film",			# Movie
	81: "film",			# Documentary
	82: "film",			# Episodic
	83: "film",			# Short
	84: "film",			# Tutorial
}

In [66]:
count = 0
for i in range(df.shape[0]):
    for genre_id in df.loc[i, "genres"]:
        key = genre_code.get(genre_id, "")
        if len(key) != "":
            df.loc[i, key] = True
    if count % 10000 == 0:
        print(f"Step {count} done")
    count += 1

df[["tool", "nsfw", "film"]]

Step 0 done
Step 10000 done
Step 20000 done
Step 30000 done
Step 40000 done
Step 50000 done
Step 60000 done
Step 70000 done
Step 80000 done
Step 90000 done
Step 100000 done
Step 110000 done


,tool,nsfw,film
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,True,False,False
...,...,...,...
115965,False,False,True
115966,False,False,True
115967,False,False,True
115968,False,False,True


In [67]:
df[["tool", "nsfw", "film"]].sum()

tool    3140
nsfw     716
film     241
dtype: int64

In [68]:
df.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'fullgame',
       'packages', 'genres', 'demos', 'platform_windows', 'platform_mac',
       'platform_linux', 'storage_mb', 'memory_mb', 'release_year',
       'release_quarter', 'coming_soon', 'package_number', 'currency',
       'initial_price', 'final_price', 'discount_percent', 'developers_amount',
       'publishers_amount', 'single', 'multi', '', 'support_vr',
       'support_controller', 'lang_en', 'lang_fr', 'lang_de', 'lang_es',
       'lang_po', 'lang_zh', 'lang_ja', 'lang_ko', 'lang_it', 'lang_ru',
       'lang_ar', 'tool', 'nsfw', 'film'],
      dtype='object')

## Done!!!

___

In [69]:
df["demos"].isna().sum()

107915

In [70]:
df[['type', 'name', 'steam_appid', 'required_age', 'is_free', 'packages',
       'genres', 'demos', 'platform_windows', 'platform_mac', 'platform_linux',
       'storage_mb', 'memory_mb', 'release_year', 'release_quarter', 'coming_soon',
       'package_number']].head()

,type,name,steam_appid,required_age,is_free,packages,genres,demos,platform_windows,platform_mac,platform_linux,storage_mb,memory_mb,release_year,release_quarter,coming_soon,package_number
0,demo,Pin Them Demo,1904630.0,0,True,NaN,[],NaN,True,False,False,NaN,NaN,2023,2.0,False,0
1,game,Al-Qadim: The Genie's Curse,1904640.0,0,False,[685932],"[1, 3]",NaN,True,False,False,2048.0,512.0,2022,1.0,False,1
2,game,Dungeons & Dragons - Stronghold: Kingdom Simul...,1904650.0,0,False,[685935],"[28, 2]",NaN,True,False,False,2048.0,512.0,2022,1.0,False,1
3,game,Chapel 3-D: The Ascent,1904680.0,0,False,NaN,"[1, 23]",NaN,True,False,False,100.0,1024.0,None,NaN,True,0
4,game,VTuber Gallery : Anime Pose,1904690.0,0,True,NaN,"[51, 53, 55, 57, 59, 70]",NaN,True,False,False,4.0,8192.0,2022,1.0,False,0


In [71]:
df[['currency', 'initial_price', 'final_price',
       'discount_percent', 'single', 'multi', '', 'support_vr',
       'support_controller', 'lang_en', 'lang_fr', 'lang_de', 'lang_es',
       'lang_po', 'lang_zh', 'lang_ja', 'lang_ko', 'lang_it', 'lang_ru',
       'lang_ar', 'tool', 'nsfw']]

,currency,initial_price,final_price,discount_percent,single,multi,,support_vr,support_controller,lang_en,...,lang_es,lang_po,lang_zh,lang_ja,lang_ko,lang_it,lang_ru,lang_ar,tool,nsfw
0,None,NaN,NaN,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,VND,8000000.0,8000000.0,0.0,True,False,True,False,False,True,...,True,False,False,False,False,False,False,False,False,False
2,VND,8000000.0,8000000.0,0.0,True,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,None,NaN,NaN,NaN,True,False,True,False,False,True,...,True,True,False,False,False,False,False,False,False,False
4,None,NaN,NaN,NaN,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115965,None,NaN,NaN,NaN,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
115966,None,NaN,NaN,NaN,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
115967,None,NaN,NaN,NaN,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
115968,None,NaN,NaN,NaN,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False


In [72]:
df.drop(columns=["", "demos", "packages", "fullgame"], inplace=True)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115970 entries, 0 to 115969
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   type                115970 non-null  object 
 1   name                115956 non-null  object 
 2   steam_appid         115970 non-null  float64
 3   required_age        115970 non-null  int64  
 4   is_free             115970 non-null  bool   
 5   genres              115970 non-null  object 
 6   platform_windows    115970 non-null  bool   
 7   platform_mac        115970 non-null  bool   
 8   platform_linux      115970 non-null  bool   
 9   storage_mb          87119 non-null   float64
 10  memory_mb           87338 non-null   float64
 11  release_year        103399 non-null  object 
 12  release_quarter     101681 non-null  float64
 13  coming_soon         115970 non-null  bool   
 14  package_number      115970 non-null  int64  
 15  currency            73018 non-null

In [74]:
df.to_csv("archive/cleaned_steam_db_v1.csv")